# Feelpp NIRB Online

# Import useful modules 

In [ ]:
import sys
import os
### uncomment above two line ones to add the feelpp path with nirb module in the python environement
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
from feelpp.mor.nirb.nirb import *
from feelpp.mor.nirb.utils import init_feelpp_environment
from feelpp.mor.nirb.nirb_perf import ComputeErrorSampling, ComputeErrors
import time
import json
import pandas as pd

## Set configuration

In [ ]:
import IPython
module_locals = IPython.extract_module_locals()[1]
# get directory of the notebook, only working if the notebook is run in vscode
if "__vsc_ipynb_file__" in module_locals:
    model_dir = "/".join(module_locals["__vsc_ipynb_file__"].split("/")[:-1]) + "/model"
# if not, we need to hard code the path...
else:
    model_dir = "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model"

Choose config file

In [ ]:
config_file = os.path.join(model_dir, "square/square.cfg")
N = None # number of basis function saved
outdir = None

Set feelpp environnement 

In [ ]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType']
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'])
config_nirb = feelpp.readJson(nirb_file)['nirb']

Get number of basis functions saved. By defaults, it will take the value filled in the model path. 

In [ ]:
nbSnap = N
if nbSnap == None:
    nbSnap = config_nirb['nbSnapshots']

Set the input/output path 

In [ ]:
# doGreedy = config_nirb['greedy-generation']
doGreedy = False
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

if outdir is None:
    RESPATH = f"results/{rectPath}/{greedyPath}"
else:
    RESPATH = outdir

Initialize nirb offline class 

In [ ]:
nirb_on = nirbOnline(**config_nirb)
nirb_on.initModel()

## Load offline data

In [ ]:
err = nirb_on.loadData(path=RESPATH, nbSnap=nbSnap)
assert err == 0, "Error while loading data"

Choose the parameter to cumpute nirb approximate solution 

In [ ]:
mu = nirb_on.Dmu.mumin() # get the min value of parameters given in the model
mu.view()

Compute approximate solution 

In [ ]:
uHh = nirb_on.getOnlineSol(mu)
uh = nirb_on.getToolboxSolution(nirb_on.tbFine, mu)

In [ ]:
l2Mat = nirb_on.generateOperators()
print(nirb_on.normMat(uHh - uh, l2Mat))
print((uHh - uh).linftyNorm())

In [ ]:
exporterName = ["ExporterNoRect", "ExporterRect"][doRectification]
nirb_on.initExporter("exporterName")
nirb_on.exportField(uHh, "uHh")
nirb_on.exportField(uh, "uh")
nirb_on.saveExporter()

Get solution for different size of subspaces

In [ ]:
solutions = []
for n in range(1, nirb_on.N+1):
    solutions.append(nirb_on.getOnlineSol(mu, n))

In [ ]:
nirb_on.initExporter("NirbN")
nirb_on.exportField(uh, "uh")
for i, uHnN in enumerate(solutions):
    nirb_on.exportField(uHnN, f"uHn{i+1:02}")
nirb_on.saveExporter()

# NIRB perfs

We can compare the above nirb solution with the FE one.

In [ ]:
errorN = ComputeErrors(nirb_on, mu, h1=True)
df = pd.DataFrame(errorN)
df['N'] = nirb_on.N

### Display errors 

In [ ]:
if feelpp.Environment.isMasterRank():
    print(f"[NIRB online] with {nirb_on.N} basis functions ")
    print(f"[NIRB online] Convergence error between nirb and FE solution ")
    print(df.head())

Or get error on a sampling of parameters.
This will compare the NIRB solution with the FE one on the fine mesh computed from parameter in the sampling (takes little time).

In [ ]:
Nsample = 30
errorN = ComputeErrorSampling(nirb_on, Nsample=Nsample, h1=True)
df = pd.DataFrame(errorN)
df['N'] = nirb_on.N

### Display errors 

In [ ]:
if feelpp.Environment.isMasterRank():
    print(f"[NIRB online] with {nirb_on.N} basis functions ")
    print(f"[NIRB online] Error on a sampling of {df.shape[0]} parameters ")
    data_mean = df.mean(axis=0)
    print("[NIRB online] Mean of errors ")
    print(data_mean)
    data_min = df.min(axis=0)
    print("[NIRB online] Min of errors ")
    print(data_min)
    data_max = df.max(axis=0)
    print("[NIRB online] Max of errors ")
    print(data_max)

Save covergence error values in a `csv` file for making statistics or vizualisation   

In [ ]:
file = os.path.join(RESPATH, f"errors{Nsample}Params.csv")
header = not os.path.isfile(file)
df.to_csv(file, mode='a', index=False, header=header)